<a href="https://colab.research.google.com/github/BPALAN-USD/AAI-502/blob/main/AAI_520_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **1. Importing all the required libraries**

In [1]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups


### **2. Display all the Categories first to filter out the required one's**

In [2]:
# Load full dataset (just metadata)
newsgroups_all = fetch_20newsgroups(subset='all')

# Show available categories
print("Available categories:\n")
for cat in newsgroups_all.target_names:
    print(cat)

Available categories:

alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
soc.religion.christian
talk.politics.guns
talk.politics.mideast
talk.politics.misc
talk.religion.misc


### **3. Fiter for only 5 categories: Business, Entertainment, Politics, Sport, Tech**

In [3]:
# Define categories similar to BBC dataset
categories = ['talk.politics.misc', 'rec.sport.baseball', 'sci.electronics',
              'comp.sys.mac.hardware', 'misc.forsale']

# Fetch dataset
newsgroups = fetch_20newsgroups(subset='all', categories=categories,
                                remove=('headers', 'footers', 'quotes'))

# Convert to DataFrame
df = pd.DataFrame({
    'text': newsgroups.data,
    'category': [newsgroups.target_names[i] for i in newsgroups.target]
})

print(df.head())
print(df['category'].value_counts())


                                                text            category
0  I have 5 full reels of Ampex 456 2" recording ...        misc.forsale
1                                                ...     sci.electronics
2  \nNo.  I do not find this conceivable.  I also...  rec.sport.baseball
3          I am looking for a large futon and frame.        misc.forsale
4  I have the following games for sale or trade f...        misc.forsale
category
rec.sport.baseball       994
sci.electronics          984
misc.forsale             975
comp.sys.mac.hardware    963
talk.politics.misc       775
Name: count, dtype: int64
